In [ ]:
import subprocess
import neutralb1.utils as utils

WORKSPACE_DIR = utils.get_workspace_dir()

git_hash = subprocess.check_output(['git', 'rev-parse', 'HEAD'], cwd=WORKSPACE_DIR).decode('utf-8').strip()
print(git_hash)

# Fits to Data With Spin-1 D/S-Constrained Waves and Both Reflectivities
Studied here are fits to data with a spin-1 waveset, where the D/S ratio and phase are constrained. We fit with both reflectivities as the unnatural component seems significant in the b1 region. All spin-projections $m$ are allowed Details of the waveset are

* Only spin-1 ($J=1$) waves
  * {$J^P\ell$} = {$1^+S, 1^-P, 1^+D$}
* The D and S waves are constrained to each other, such that every $D$-wave has a common ratio and phase between its $S$-wave partner
  * Boundaries: $0< r_{D/S} < 1,\quad -\pi<\phi_{D-S}<\pi$
* Include an isotropic background term
  * Assumed that sideband subtraction handles almost all background events, and so these events will be *true* $\omega$ events, but contribute in total a flat distribution to the angular distributions
* Uses all 4 polarization orientations
* Split data into bins of mass and $-t$
  * 20 MeV wide bins from $1.0 < M_{\omega\pi^0} < 1.8$ 
* Perform 500 randomized fits + 100 bootstrap fits in each bin of mass and $-t$ independently

## Setup

In [ ]:
# load common libraries
import pandas as pd
import pickle as pkl
import pathlib
import os, sys
import numpy as np
import matplotlib.pyplot as plt

# load neutralb1 libraries
import neutralb1.utils as utils
from neutralb1.analysis.result import ResultManager

utils.load_environment()

# load in useful directories as constants
CWD = pathlib.Path.cwd()
STUDY_DIR = f"{WORKSPACE_DIR}/studies/data-fits/spin-1/constrained-ratio/"

# set env variables for shell cells
os.environ["WORKSPACE_DIR"] = WORKSPACE_DIR
os.environ['STUDY_DIR'] = STUDY_DIR

In [ ]:
%%bash
# print out yaml file used to submit the fits
cat $STUDY_DIR/submission.YAML

In [ ]:
# load in the preprocessed results 
t_bins = ["t_0.10-0.16", "t_0.16-0.23", "t_0.23-0.35", "t_0.35-1.00"]
binned_results = {}
for t_bin in t_bins:
    with open(f"{STUDY_DIR}/{t_bin}/preprocessed_results_acceptance_corrected.pkl", "rb") as f:
        data = pkl.load(f)
        binned_results[t_bin] = ResultManager(**data)

binned_results: dict[str, ResultManager]

In [ ]:
for t_bin, results in binned_results.items():
    print(f"Results for t bin: {t_bin}")
    results.summary()

## Analysis

### Standard Plots
These plots are always handy to have

In [ ]:
fig, axs = plt.subplots(1, 4, figsize=(30, 6))
for i, (t_bin, results) in enumerate(binned_results.items()):    
    results.plot.intensity.jp(ax=axs[i])
    t_low, t_high = results.get_t_edges()    
    axs[i].set_title(rf"${t_low} \leq -t \leq {t_high}$ GeV²")
    if i != 0:
        axs[i].set_ylabel("")
        axs[i].get_legend().remove()
    
plt.savefig(f"{STUDY_DIR}/jp_all_t_bins.pdf")    

In [ ]:
for t_bin, results in binned_results.items():
    results.plot.intensity.waves()
    t_low, t_high = results.get_t_edges()
    utils.big_print(rf"{t_low} < -t < {t_high} GeV²", 2.0)
    plt.savefig(f"{STUDY_DIR}/{t_bin}/plots/waves.pdf")  
    plt.show()

In [ ]:
for t_bin, results in binned_results.items():
    results.plot.intensity.waves(fractional=True)
    t_low, t_high = results.get_t_edges()
    utils.big_print(rf"{t_low} < -t < {t_high} GeV²", 2.0)
    plt.savefig(f"{STUDY_DIR}/{t_bin}/plots/waves_fractional.pdf")  
    plt.show()

In [ ]:
for t_bin, results in binned_results.items():
    results.plot.intensity.waves(reflectivity="negative")
    t_low, t_high = results.get_t_edges()
    utils.big_print(rf"{t_low} < -t < {t_high} GeV²", 2.0)
    plt.savefig(f"{STUDY_DIR}/{t_bin}/plots/waves_neg.pdf")  
    plt.show()

In [ ]:
for t_bin, results in binned_results.items():
    results.plot.diagnostic.matrix()
    t_low, t_high = results.get_t_edges()
    utils.big_print(rf"{t_low} < -t < {t_high} GeV²", 2.0)
    plt.savefig(f"{STUDY_DIR}/{t_bin}/plots/matrix.pdf") 
    plt.show()

In [ ]:
sig_moments = set()
for t_bin, results in binned_results.items():    
    sig_moments = sig_moments.union(results.get_significant_moments(threshold=0.04))

fig, axs = plt.subplots(
    len(sig_moments), 4, figsize=(20, 25), layout="constrained", sharex=True, sharey="row"
)
for col, (t_bin, results) in enumerate(binned_results.items()):    
    for row, moment in enumerate(sig_moments):        
        ax = axs[row, col]
        label = utils.convert_moment_name(moment) if col == 0 else ""
        my_kwargs = {moment: {"label": label}}
        results.plot.intensity.plot([moment], ax=ax, col_kwargs=my_kwargs)
        if col != 0:
            ax.set_ylabel("")
    t_low, t_high = results.get_t_edges()
    axs[0,col].set_title(rf"${t_low} \leq -t \leq {t_high}$ GeV$^2$")

plt.savefig(f"{STUDY_DIR}/significant_moments_all_t_bins.pdf")


### D/S Ratio and Phase

In [ ]:
fig, axs = plt.subplots(
    3,
    4,
    sharex=True,
    sharey="row",
    figsize=(20, 10),
    gridspec_kw={"wspace": 0.0, "hspace": 0.12},
    layout="constrained",
)
for i, (t_bin, results) in enumerate(binned_results.items()):
    results.plot.diagnostic.ds_ratio(axs = axs[:, i])
    if i != 0:
        for ax in axs[:, i]:
            ax.get_legend().remove()
            ax.set_ylabel("")
    
    t_low, t_high = results.get_t_edges()
    axs[0,i].set_title(rf"${t_low} \leq -t \leq {t_high}$ GeV²")
plt.savefig(f"{STUDY_DIR}/ds_ratio_all_t_bins.pdf")

### $b_1(1235)$ and $1^{--}$ interference

In [ ]:
fig, axs = plt.subplots(
    2,
    4,
    sharex=True,        
    gridspec_kw={"wspace": 0.0, "hspace": 0.07},
    height_ratios=[3, 1],
    figsize=(30, 10),
    layout="constrained",
)
for i, (t_bin, results)in enumerate(binned_results.items()):
    results.plot.phase.mass_phase(
        amp1="p1p0S", amp2="p1mpP",
        amp1_kwargs={"color":"tab:blue"},
        amp2_kwargs={"color":"tab:orange"},
        amp_ax=axs[0, i],
        phase_ax=axs[1, i],
    )
    t_low, t_high = results.get_t_edges()
    axs[0,i].set_title(rf"${t_low} \leq -t \leq {t_high}$ GeV$^2$", fontsize=20)

max_y=0
for i in range(4):
    y_max = axs[0,i].get_ylim()[1]
    max_y = max(max_y, y_max)
for i in range(4):
    axs[0,i].set_ylim(0, max_y)


plt.savefig(f"{STUDY_DIR}/mass_phase_all_t_bins.pdf")

Lets make a few joyplots to see how their bootstrap distributions are shaped

In [ ]:
# they have to be made individually because the joyplot is a bunch of axes, so we can't easily put them all in one figure
for i, (t_bin, results)in enumerate(binned_results.items()):
    t_low, t_high = results.get_t_edges()
    utils.big_print(f"{t_low} < -t < {t_high}", 2.0)
    results.plot.bootstrap.joyplot(
        ["p1p0S", "p1mpP"]        
    )
    plt.savefig(f"{STUDY_DIR}/{t_bin}/plots/joyplot_p1p0S_p1mpP.pdf")
    plt.show()    

In [ ]:
for i, (t_bin, results)in enumerate(binned_results.items()):
    t_low, t_high = results.get_t_edges()
    utils.big_print(f"{t_low} < -t < {t_high}", 2.0)
    phase = results.phase_difference_dict[("p1p0S", "p1mpP")]
    results.plot.bootstrap.joyplot(
        [phase]
    )
    plt.savefig(f"{STUDY_DIR}/{t_bin}/plots/joyplot_p1p0S_p1mpP.pdf")
    plt.show()    

#### P-wave amplitude bootstraps
The bootstrap distributions in the 1.4-1.5 region of the $1^-P_{+1}^{(+)}$ amplitude look thin. Lets investigate this region further

In [ ]:
fit_indices = binned_results["t_0.10-0.16"].get_fit_indices(mass_low=1.4, mass_high=1.5)
fit_indices

In [ ]:
cols = set()
for i, (t_bin, results) in enumerate(binned_results.items()):
    cols = cols.union(results.get_significant_amplitudes(fit_indices=fit_indices))

cols = ["m1mmP", "m1mpP", "p1mmP", "p1mpP"]
for i, (t_bin, results) in enumerate(binned_results.items()):
    t_low, t_high = results.get_t_edges()
    utils.big_print(f"{t_low} < -t < {t_high}", 2.0)
    results.plot.bootstrap.pairplot(
        fit_indices,
        list(sorted(cols))
    )
    plt.show()

Seems that there may be possible leakage from dominant m=+1 wave to the m=-1 wave in either reflectivity

### $1^-P_0^{(\varepsilon)}$ Wave
There's a possibility for this wave to have a $\rho(1700)$ contribution. No notable phase motion is seen in this wave, but lets take a closer look at the amplitudes to be sure

In [ ]:
fig, axs = plt.subplots(
    1, 4, figsize=(20, 5), layout="constrained", sharex=True,
)
for col, (t_bin, results) in enumerate(binned_results.items()):
    ax = axs[col]
    my_kwargs = {
        "p1m0P": {"color": "tab:red"},
        "m1m0P": {"color": "tab:blue"},
    }
    results.plot.intensity.plot(["p1m0P", "m1m0P"], ax=ax, col_kwargs=my_kwargs)
    t_low, t_high = results.get_t_edges()
    ax.set_title(rf"${t_low} \leq -t \leq {t_high}$ GeV$^2$")

plt.savefig(f"{STUDY_DIR}/1m0P_all_t_bins.pdf")

### Naturalities

In [ ]:
fig, axs = plt.subplots(
    4, 4, figsize=(20, 15), layout="constrained", sharex=True, sharey="row"
)

for col, (t_bin, results) in enumerate(binned_results.items()):
    row_values = ["p1p", "p1m", "m1p", "m1m"]
    labels = ["Natural $1^+$", "Natural $1^-$", "Unnatural $1^+$", "Unnatural $1^-$"]
    colors = ["tab:red", "tab:red", "tab:blue", "tab:blue"]
    for row in range(4):
        ax = axs[row, col]    
        label = labels[row] if col == 0 else ""
        val = row_values[row]
        my_kwargs = {val: {"label": label, "color": colors[row]}}
        results.plot.intensity.plot([val], ax=ax, col_kwargs=my_kwargs)

plt.savefig(f"{STUDY_DIR}/naturalities_all_t_bins.pdf")

### Simple Breit-Wigner Line Shape Fits to S-waves

In [ ]:
from scipy.optimize import curve_fit
from neutralb1.analysis.physics import breit_wigner

vec_bw = np.vectorize(breit_wigner)
function_masses = np.arange(1.0, 1.5, 0.001)

def bw_model(mass, scale, bw_mass, bw_width):
    bw = vec_bw(mass, bw_mass=bw_mass, bw_width=bw_width, bw_l=0)    
    return scale * np.square(np.abs(bw))

fig, axs = plt.subplots(
    3, 4, figsize=(20, 10), layout="constrained", sharey=True, sharex=True
)

for col, (t_bin, results) in enumerate(binned_results.items()):
    row_y_titles = ["$1^+S_{1}^{(+)}$", "$1^+S_{0}^{(+)}$", "$1^+S_{-1}^{(+)}$"]
    row_wave = ["p1ppS", "p1p0S", "p1pmS"]
    for row in range(3):                
        data_masses = results.get_mass_centers()
        data_values = results.fit_df[row_wave[row]].values
        data_errors = results.plot.intensity.get_bootstrap_error(row_wave[row])

        # determine where data_mass > 1.5 GeV
        fit_masses = [m for m in data_masses if m < 1.5]
        fit_values = data_values[:len(fit_masses)] 
        fit_errors = data_errors[:len(fit_masses)]       

        popt, pcov = curve_fit(
            bw_model,
            fit_masses,
            fit_values,
            sigma=fit_errors,
            p0=[200000, 1.23, 0.14],
        )
        perr = np.sqrt(np.diag(pcov))
        scale_opt, mass_opt, width_opt = popt
        fitted_curve = bw_model(function_masses, *popt)

        ax = axs[row, col]
        ax.plot(function_masses, fitted_curve, color="red", linestyle="-", linewidth=2, label="Fitted BW")
        ax.errorbar(
            x=data_masses, 
            y=data_values, 
            yerr=data_errors, 
            linestyle="", 
            marker=".", 
            color="black",             
        )
        
        ax.set_ylabel(row_y_titles[row] if col == 0 else "", loc="center", fontsize=20)

        # Plot PDG shape for comparison
        pdg_mass = 1.2295
        pdg_width = 0.142
        pdg_curve = bw_model(function_masses, scale_opt, pdg_mass, pdg_width)
        ax.plot(function_masses, pdg_curve, color="blue", linestyle="--", linewidth=2, alpha=0.6, label="PDG BW")
        
        if row == 0:
            t_low, t_high = results.get_t_edges()
            ax.set_title(rf"${t_low} \leq -t \leq {t_high}$ GeV$^2$", fontsize=20)    
        if row == 2:
            ax.set_xlabel(r"$\omega\pi^0$ inv. mass (GeV)")
        
        textstr = (
            f"M: {mass_opt:.3f} ± {perr[1]:.3f} GeV\n"
            rf"$\Gamma$: {width_opt:.3f} ± {perr[2]:.3f} GeV"
        )
        ax.text(0.95, 0.95, textstr, transform=ax.transAxes, fontsize=15,
                verticalalignment='top', horizontalalignment='right',
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))

plt.savefig(f"{STUDY_DIR}/BW-b1_refl+_all_t_bins.pdf")


In [ ]:
from scipy.optimize import curve_fit
from neutralb1.analysis.physics import breit_wigner

vec_bw = np.vectorize(breit_wigner)
function_masses = np.arange(1.0, 1.5, 0.001)

def bw_model(mass, scale, bw_mass, bw_width):
    bw = vec_bw(mass, bw_mass=bw_mass, bw_width=bw_width, bw_l=0)    
    return scale * np.square(np.abs(bw))

fig, axs = plt.subplots(
    3, 4, figsize=(20, 10), layout="constrained", sharey=True, sharex=True
)

for col, (t_bin, results) in enumerate(binned_results.items()):
    row_y_titles = ["$1^+S_{1}^{(-)}$", "$1^+S_{0}^{(-)}$", "$1^+S_{-1}^{(-)}$"]
    row_wave = ["m1ppS", "m1p0S", "m1pmS"]
    for row in range(3):                
        data_masses = results.get_mass_centers()
        data_values = results.fit_df[row_wave[row]].values
        data_errors = results.plot.intensity.get_bootstrap_error(row_wave[row])

        # determine where data_mass > 1.5 GeV
        fit_masses = [m for m in data_masses if m < 1.5]
        fit_values = data_values[:len(fit_masses)] 
        fit_errors = data_errors[:len(fit_masses)]       

        # we need to introduce bounds for negative reflecitivity waves to avoid unphysical fit results
        lower_bounds = [-np.inf, 1.1, 0.05]
        upper_bounds = [np.inf, 1.4, 0.4]

        popt, pcov = curve_fit(
            bw_model,
            fit_masses,
            fit_values,
            sigma=fit_errors,
            p0=[200000, 1.23, 0.14],
            bounds=(lower_bounds, upper_bounds)
        )
        perr = np.sqrt(np.diag(pcov))
        scale_opt, mass_opt, width_opt = popt
        fitted_curve = bw_model(function_masses, *popt)

        ax = axs[row, col]
        ax.plot(function_masses, fitted_curve, color="red", linestyle="-", linewidth=2, label="Fitted BW")
        ax.errorbar(
            x=data_masses, 
            y=data_values, 
            yerr=data_errors, 
            linestyle="", 
            marker=".", 
            color="black",             
        )
        
        ax.set_ylabel(row_y_titles[row] if col == 0 else "", loc="center", fontsize=20)

        # Plot PDG shape for comparison
        pdg_mass = 1.2295
        pdg_width = 0.142
        pdg_curve = bw_model(function_masses, scale_opt, pdg_mass, pdg_width)
        ax.plot(function_masses, pdg_curve, color="blue", linestyle="--", linewidth=2, alpha=0.6, label="PDG BW")
        
        if row == 0:
            t_low, t_high = results.get_t_edges()
            ax.set_title(rf"${t_low} \leq -t \leq {t_high}$ GeV$^2$", fontsize=20)    
        if row == 2:
            ax.set_xlabel(r"$\omega\pi^0$ inv. mass (GeV)")
        
        textstr = (
            f"M: {mass_opt:.3f} ± {perr[1]:.3f} GeV\n"
            rf"$\Gamma$: {width_opt:.3f} ± {perr[2]:.3f} GeV"
        )
        ax.text(0.95, 0.95, textstr, transform=ax.transAxes, fontsize=15,
                verticalalignment='top', horizontalalignment='right',
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))

plt.savefig(f"{STUDY_DIR}/BW-b1_refl-_all_t_bins.pdf")

### Discontinuity Pairplots
Look at the pairplots for before/after the discontinuity in each t bin 

In [ ]:
bin_to_discontinuity_mass_location = {
    "t_0.10-0.16": [1.42, 1.46],
    "t_0.16-0.23": [1.40, 1.44],
    "t_0.23-0.35": [1.42, 1.46],
}

for i, (t_bin, results) in enumerate(binned_results.items()):
    if i == 3:
        continue # ignore last t bin, no discontinuity there
    m_low, m_high = bin_to_discontinuity_mass_location[t_bin]
    fit_indices = results.get_fit_indices(m_low, m_high)
    t_low, t_high = results.get_t_edges()
    utils.big_print(f"{t_low} < -t < {t_high}", 2.0)
    results.plot.bootstrap.pairplot(
        fit_indices,
        ["p1pmS", "p1p0S", "p1ppS", "p1pmD", "p1p0D", "p1ppD"],        
    )
    plt.show()
    plt.savefig(f"{STUDY_DIR}/{t_bin}/plots/pairplot_discontinuity.pdf")